# EXCEED Python Qualtrics Survey - Detailed Analysis

This notebook provides a detailed analysis of the EXCEED Python Qualtrics survey data. It includes data loading, cleaning, and various analyses to understand the survey responses better. The ultimate goal of this analysis is to find the best subset of questions that can best differentiate between Python _beginner_/_novice_ and _advanced_/_expert_ users.

The Qualtrics survey is composed of the following blocks, where each participant is required to answer all questions in each block. Note that for blocks 3 to 8, each respondent is given a random subset of 2 out of the 7 available questions, hence the total number of survey-related questions is 16. The blocks are as follows:
1. **Consent Form**: Participants agree to take part in the survey. If they do not agree, they are redirected to the end of the survey.
2. **Self-assessment**:
    1. **Python Experience**: Participants self-assess their Python experience using Dreyfus levels.
    2. **Python Programming YoE**: Participants indicate their years of experience with Python programming.
    3. **General Programming YoE**: Participants indicate their years of experience with programming in general.
3. **General Programming Error Understanding**: 7 questions
4. **Python-Specific General Error Understanding**: 7 questions
5. **Code Reading / Understanding**: 7 questions
6. **Error Message Comprehension**: 7 questions
7. **Error Resolution**: 7 questions
8. **Error Message Comprehension**: 7 questions
9. **Natural Language Scenarios**: 7 questions
10. **Miscellanous Questions - Various Complexity & Scope**: 7 questions
11. **Self-Assessment of Results**: Participants self-assess the number of questions they answered correctly in the survey (0 to 16).

## Step 0.1: Install Required Libraries

Prior to running the analysis, ensure that the required libraries are installed. For this, you need only install the libraries defined in the `requirements.txt` file. You can do this by running the following command in your terminal:

```bash
pip install -r requirements.txt
```

In [15]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Step 0.2: Load the data

For this step, we need to load 2 CSV files:
1. `survey_results.csv`: Contains the survey responses and some other metadata
2. `survey_answers.csv`: Contains the correct answers for the survey questions

In [16]:
import pandas as pd

file_path_survey_results = "../data/survey_results.csv"
file_path_survey_answers = "../data/survey_answers.csv"

# Step 0. Load data (files are semicolon‑delimited)
df_results = pd.read_csv(file_path_survey_results, sep=';')
df_answers = pd.read_csv(file_path_survey_answers, sep=';')

## Step 1: Build Answers DataFrame

To calculate the accuracy of the answers, we need to build a DataFrame that contains the correct answers for each question. This DataFrame will be used to compare the participants' responses against the correct answers. This step will create a dictionary that maps each question ID to its correct answer, and then build a list of question columns that are present in the results DataFrame.

In [17]:
answer_key = dict(zip(df_answers["QuestionId"], df_answers["CorrectAnswer"]))

# 1. Build columns for each question
question_cols = [q for q in answer_key if q in df_results.columns]

## Step 2: Mark Questions as Correct or Incorrect

To determine whether a participant answered a question correctly, we will compare their response to the correct answer. We will create a new DataFrame that contains the correctness of each answer (True for correct, False for incorrect).

In [18]:
# 2. Create a new DataFrame with the correct answers
correct_df = df_results[question_cols].eq(pd.Series(answer_key))
df_results["total_correct"] = correct_df.sum(axis=1)

## Step 3: Compute Accuracy for Each Question

Compute the accuracy for each question by calculating the percentage of correct answers among all participants who answered that question. This will help us identify which questions are more effective in differentiating between beginner/novice and advanced/expert users. This step will create a DataFrame that contains the percentage of correct answers for each question, sorts them in descending order of correctness, and displays the results in a readable format.

In [19]:
# Boolean mask: True if answered AND correct, NaN if unanswered
correct_mask = (
    df_results[question_cols]
    .eq(pd.Series(answer_key))
    .where(df_results[question_cols].notna())
)

correct_mask = correct_mask.astype("float")

# Per-question counts
correct_cnt = correct_mask.sum()
answered_cnt = df_results[question_cols].notna().sum()

# Calculate the percentage of correct answers for each question
perc_correct = (correct_cnt / answered_cnt * 100).round(1).rename("% correct")

# Create a DataFrame with the percentage of correct answers
question_perf = (
    perc_correct.reset_index()
    .rename(columns={"index": "Question"})
    .sort_values("% correct", ascending=False)
    .reset_index(drop=True)
)

# Display the question performance DataFrame
from IPython.display import display, HTML
display(HTML("<h2>Question Performance / Difficulty</h2>"))
display(question_perf)

# Save the question performance DataFrame to a CSV file for further analysis
from pathlib import Path
out_dir = Path("../data/processed")
out_dir.mkdir(parents=True, exist_ok=True)
question_perf.to_csv(out_dir / "question_performance.csv", index=False)

,Question,% correct
0,Q8.7,100.0
1,Q8.5,100.0
2,Q7.5,100.0
3,Q6.3,100.0
4,Q6.1,100.0
5,Q8.3,100.0
6,Q5.6,100.0
7,Q7.4,100.0
8,Q10.6,100.0
9,Q10.4,100.0


## Step 4: Calculate Question (i.e., item) Difficulty and Discrimination

We will attempt to find the top 2 "_best_" questions from each block based on their accuracy. The criteria for "_best_" is a combination of item difficulty and item discrimination, which can be calculated using the following metrics:

1. **Item Difficulty**: The percentage of correct answers for a given question (already calculated in the previous step).
2. **Item Discrimination**: The correlation with a respondent's overall ability.

This is a pre-requisite for actually finding the best subset of questions, which will be done in the next step. Additionally, this step will also skip any items that have fewer than 10 valid answers (i.e., completed) to avoid unstable discrimination index calculations. To the best of our knowledge, such questions do not exist in the current dataset, but this is a good practice to follow in general.

> Note: When running the code below, you may see some warnings similar to `ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.` - this is expected, and can be ignored. It simply means that the question was answered the same way by all respondents, so there is no variability to compute a correlation.

In [20]:
from scipy.stats import pointbiserialr

# Total score per respondent (all questions from block 3 to block 10 inclusive); NaN treated as 0
df_results["total_score"] = correct_mask.fillna(0).sum(axis=1)

# Compute p (proportion correct) and r_pb (discrimination index)
records = []

for q in correct_mask.columns:
    # Boolean / NaN vector for this item
    item = correct_mask[q]

    # Skip items with < 10 valid answers to avoid unstable r
    if item.notna().sum() < 10:
        continue

    # Difficulty p
    p = item.mean(skipna=True)

    # Rest-of-test score (subtract the item itself if answered)
    rest = df_results["total_score"] - item.fillna(0)

    # Point-biserial correlation (returns r, p-value)
    r_pb, _ = pointbiserialr(item.dropna(), rest.loc[item.notna()])

    block = q.split(".")[0]
    records.append(
        {
            "block": block,
            "question": q,
            "p": p,
            "r_pb": r_pb
        }
    )

# Create a DataFrame with the calculated statistics
stats_df = pd.DataFrame(records)

# Save the statistics DataFrame to a CSV file for further analysis
stats_df.to_csv(out_dir / "item_stats.csv", index=False)

/Users/amoraru/Documents/GitHub/EXCEED/exceed-python-skill-assessment/.venv/lib/python3.12/site-packages/scipy/stats/_stats_py.py:5405: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  rpb, prob = pearsonr(x, y)


## Step 5: Filter Questions for Acceptable Difficulty Range & Display All Items

This step filters the questions based on the acceptable difficulty range (0.25 <= p <= 0.85) and displays all items ranked by their discrimination index (r_pb). This will help us understand which questions are more effective in differentiating between beginner/novice and advanced/expert users. In terms of output, this step does the following:
- Displays the number of survey questions considered before filtering for acceptable difficulty range. This number should be 56, since there are 7 questions in each of the 8 blocks (3 to 10).
- Displays the number of survey questions considered after filtering for acceptable difficulty range.

> Note: This is still not the final subset of questions, but rather a preliminary step to identify the best questions based on their difficulty and discrimination index. On top of that, we will also vary the actual difficulty range in follow-up steps to find the best subset of questions that can differentiate between beginner/novice and advanced/expert users.

In [21]:
# Keep items in an acceptable difficulty range (0.25 <= p <= 0.85) - note that we will vary this range later as well
print("Number of survey questions considered before filtering for acceptable difficulty range:", len(stats_df))
filtered_stats_df = stats_df.query("0.25 <= p <= 0.85").copy()
print("Number of survey questions considered after filtering for acceptable difficulty range:", len(filtered_stats_df))

# Add a numeric block column
filtered_stats_df["block_num"] = filtered_stats_df["block"].str.slice(1).astype(int)

# Rank by discrimination index (r_pb) and take the top 2 per block
filtered_stats_df = (
    filtered_stats_df.sort_values(["block_num", "r_pb"], ascending=[True, False])
    .assign(rank=lambda d: d.groupby("block_num").cumcount() + 1)
)

# Display all the items ranked by discrimination
display(HTML("<h2>Item Statistics (after filtering for acceptable difficulty range)</h2>"))
display(filtered_stats_df[["block", "question", "p", "r_pb"]]
        .style.format({"p": "{:.2f}", "r_pb": "{:.2f}"}))

Number of survey questions considered before filtering for acceptable difficulty range: 56
Number of survey questions considered after filtering for acceptable difficulty range: 24


,block,question,p,r_pb
3,Q3,Q3.4,0.78,0.05
6,Q3,Q3.7,0.84,-0.04
2,Q3,Q3.3,0.56,-0.16
10,Q4,Q4.4,0.61,0.40
12,Q4,Q4.6,0.82,0.39
13,Q4,Q4.7,0.56,0.14
15,Q5,Q5.2,0.83,0.65
20,Q5,Q5.7,0.65,0.17
18,Q5,Q5.5,0.72,0.15
25,Q6,Q6.5,0.84,0.60


## Step 6: Find Best Subset of Questions (2 per Block)

This block will display the top 2 questions per block based on their discrimination index, i.e., how well they differentiate between beginner/novice and advanced/expert users.

### Discrimination Index (r_pb)

The discrimination index (r_pb) is calculated using the point-biserial correlation coefficient, which measures the relationship between a binary variable (correct/incorrect) and a continuous variable (total score excluding the item itself). The following ranges for r_pb are as follows:
- **0.00 - 0.19/0.2**: Weak discriminator / Should be replaced
- **0.20 - 0.29**: Acceptable discriminator
- **>= 0.30**: Strong discriminator

In [22]:
# Create a DataFrame with the top 2 questions per block based on discrimination index
top2_df = (
    filtered_stats_df[filtered_stats_df["rank"] <= 2]
    .sort_values(["block_num", "r_pb"], ascending=[True, False])
)

# Display the top 2 items per block
display(HTML("<h2>Top 2 Chosen Questions per Block</h2>"))
display(top2_df[["block", "question", "p", "r_pb"]]
        .style.format({"p": "{:.2f}", "r_pb": "{:.2f}"}))

# Save the top 2 questions per block to a CSV file for further analysis
top2_df.to_csv(out_dir / "top2_items.csv", index=False)

,block,question,p,r_pb
3,Q3,Q3.4,0.78,0.05
6,Q3,Q3.7,0.84,-0.04
10,Q4,Q4.4,0.61,0.40
12,Q4,Q4.6,0.82,0.39
15,Q5,Q5.2,0.83,0.65
20,Q5,Q5.7,0.65,0.17
25,Q6,Q6.5,0.84,0.60
22,Q6,Q6.2,0.65,0.42
29,Q7,Q7.2,0.56,0.58
30,Q7,Q7.3,0.76,0.40


## Step 7: Compute Average Correct Overall

Compute the average number of correct answers across all participants. This will give us an overall sense of how well participants performed on the survey.

In [23]:
avg_total_correct = df_results["total_correct"].mean()

## Step 8: Compute Average Correct by Block

For each of the actual survey blocks that contain skill-level assessment questions, we will compute the average correctness of the answers. This will help us understand how well participants performed in each block.

In [24]:
block_avgs = {}
for blk in range(3, 11):
    blk_cols = [c for c in question_cols if c.startswith(f"Q{blk}.")]
    block_avgs[f"Q{blk}"] = correct_df[blk_cols].sum(axis=1).mean()

## Step 9: Build Result Summary DataFrame

This DataFrame will summarize the results of the analysis, including the average correctness for each question, the overall average correctness, and the average correctness by block. This will provide a comprehensive overview of the survey results.

In [25]:
# Collect rows for the summary report
rows = [
    ("Average correct answers (all blocks)", avg_total_correct),
]

# Block averages (Q3 to Q10)
rows.extend(
    [(f"Average correct answers - Block {blk}", val) for blk, val in block_avgs.items()]
)

# Compute correlations
rows.extend([
    ("Pearson Correlation: Python YoE with total correct", df_results[["Q2.2", "total_correct"]].corr().iloc[0, 1]),
    ("Pearson Correlation: General YoE with total correct", df_results[["Q2.3", "total_correct"]].corr().iloc[0, 1]),
    ("Pearson Correlation: Total correct answers with self-reported correct answers", df_results["total_correct"].corr(df_results["Q11.1"]))
])

## Step 10: Display Result Summary

Finally, we will display the result summary DataFrame, which contains the average correctness of answers for each block and the overall correctness, as well as some additional correlation metrics.

In [26]:
# Create a DataFrame for the summary report
summary_df = pd.DataFrame(rows, columns=["Metric", "Value"])

# Display the summary report
display(
    summary_df.style.set_caption("Qualtrics Python Skill Level Assessment - Correctness Metrics").format(precision=2))

,Metric,Value
0,Average correct answers (all blocks),10.64
1,Average correct answers - Block Q3,1.46
2,Average correct answers - Block Q4,1.35
3,Average correct answers - Block Q5,1.36
4,Average correct answers - Block Q6,1.31
5,Average correct answers - Block Q7,1.28
6,Average correct answers - Block Q8,1.36
7,Average correct answers - Block Q9,1.24
8,Average correct answers - Block Q10,1.28
9,Pearson Correlation: Python YoE with total correct,0.29


## Step 11: Effect of Varying Difficulty Thresholds and Top-N Selection

Below we demonstrate how changing the difficulty thresholds and the number of top questions per block affects the final subset of discriminating questions. This allows for easy comparison and sensitivity analysis. Firstly, we will define a function to facilitate easier parameterization of the selection process, and then we will apply it with different settings to see how the results change.

In [27]:
def select_top_questions(stats_df, min_p=0.25, max_p=0.85, top_n=2):
    """
    Filter questions by difficulty and then rank them by discrimination index (i.e., the r_pb value capturing the
    relationship between the binary correctness of the question and the continuous total score excluding that question).
    Args:
        stats_df: DataFrame with columns ['block', 'question', 'p', 'r_pb']
        min_p: minimum difficulty (inclusive)
        max_p: maximum difficulty (inclusive)
        top_n: number of top questions per block to select
    Returns:
        DataFrame with top_n questions per block within the difficulty range, ranked by r_pb.
    """
    filtered = stats_df.query(f"@min_p <= p <= @max_p").copy()
    filtered["block_num"] = filtered["block"].str.slice(1).astype(int)
    filtered = (
        filtered.sort_values(["block_num", "r_pb"], ascending=[True, False])
                .assign(rank=lambda d: d.groupby("block_num").cumcount() + 1)
    )
    return filtered[filtered["rank"] <= top_n].sort_values(["block_num", "r_pb"], ascending=[True, False])

In [28]:
# Define different settings for varying difficulty thresholds and top-N selection
settings = [
    {"min_p": 0.25, "max_p": 0.85, "top_n": 2, "label": "Default Range (0.25-0.85, Top 2)"},
    {"min_p": 0.25, "max_p": 0.85, "top_n": 1, "label": "Default Range (0.25-0.85, Top 1)"},
    {"min_p": 0.20, "max_p": 0.90, "top_n": 2, "label": "Wider Range (0.20-0.90, Top 2)"},
    {"min_p": 0.20, "max_p": 0.90, "top_n": 1, "label": "Wider Range (0.20-0.90, Top 1)"},
    {"min_p": 0.20, "max_p": 0.90, "top_n": 2, "label": "Widest Range (0.20-0.95, Top 2)"},
    {"min_p": 0.20, "max_p": 0.90, "top_n": 1, "label": "Widest Range (0.20-0.95, Top 1)"},
    {"min_p": 0.30, "max_p": 0.80, "top_n": 2, "label": "Narrower Range (0.30-0.75, Top 2)"},
    {"min_p": 0.30, "max_p": 0.80, "top_n": 1, "label": "Narrower Range (0.30-0.75, Top 1)"},
]

# Display top questions for different settings
for s in settings:
    result = select_top_questions(stats_df, min_p=s["min_p"], max_p=s["max_p"], top_n=s["top_n"])
    display(HTML(f"<h4>{s['label']}</h4>"))
    display(result[["block", "question", "p", "r_pb"]].style.format({"p": "{:.2f}", "r_pb": "{:.2f}"}))


,block,question,p,r_pb
3,Q3,Q3.4,0.78,0.05
6,Q3,Q3.7,0.84,-0.04
10,Q4,Q4.4,0.61,0.40
12,Q4,Q4.6,0.82,0.39
15,Q5,Q5.2,0.83,0.65
20,Q5,Q5.7,0.65,0.17
25,Q6,Q6.5,0.84,0.60
22,Q6,Q6.2,0.65,0.42
29,Q7,Q7.2,0.56,0.58
30,Q7,Q7.3,0.76,0.40


,block,question,p,r_pb
3,Q3,Q3.4,0.78,0.05
10,Q4,Q4.4,0.61,0.40
15,Q5,Q5.2,0.83,0.65
25,Q6,Q6.5,0.84,0.60
29,Q7,Q7.2,0.56,0.58
36,Q8,Q8.2,0.78,0.43
43,Q9,Q9.2,0.61,0.05
53,Q10,Q10.5,0.78,0.37


,block,question,p,r_pb
3,Q3,Q3.4,0.78,0.05
6,Q3,Q3.7,0.84,-0.04
10,Q4,Q4.4,0.61,0.40
12,Q4,Q4.6,0.82,0.39
15,Q5,Q5.2,0.83,0.65
20,Q5,Q5.7,0.65,0.17
25,Q6,Q6.5,0.84,0.60
22,Q6,Q6.2,0.65,0.42
28,Q7,Q7.1,0.88,0.79
29,Q7,Q7.2,0.56,0.58


,block,question,p,r_pb
3,Q3,Q3.4,0.78,0.05
10,Q4,Q4.4,0.61,0.40
15,Q5,Q5.2,0.83,0.65
25,Q6,Q6.5,0.84,0.60
28,Q7,Q7.1,0.88,0.79
36,Q8,Q8.2,0.78,0.43
44,Q9,Q9.3,0.88,0.20
49,Q10,Q10.1,0.87,0.39


,block,question,p,r_pb
3,Q3,Q3.4,0.78,0.05
6,Q3,Q3.7,0.84,-0.04
10,Q4,Q4.4,0.61,0.40
12,Q4,Q4.6,0.82,0.39
15,Q5,Q5.2,0.83,0.65
20,Q5,Q5.7,0.65,0.17
25,Q6,Q6.5,0.84,0.60
22,Q6,Q6.2,0.65,0.42
28,Q7,Q7.1,0.88,0.79
29,Q7,Q7.2,0.56,0.58


,block,question,p,r_pb
3,Q3,Q3.4,0.78,0.05
10,Q4,Q4.4,0.61,0.40
15,Q5,Q5.2,0.83,0.65
25,Q6,Q6.5,0.84,0.60
28,Q7,Q7.1,0.88,0.79
36,Q8,Q8.2,0.78,0.43
44,Q9,Q9.3,0.88,0.20
49,Q10,Q10.1,0.87,0.39


,block,question,p,r_pb
3,Q3,Q3.4,0.78,0.05
2,Q3,Q3.3,0.56,-0.16
10,Q4,Q4.4,0.61,0.40
13,Q4,Q4.7,0.56,0.14
20,Q5,Q5.7,0.65,0.17
18,Q5,Q5.5,0.72,0.15
22,Q6,Q6.2,0.65,0.42
27,Q6,Q6.7,0.41,0.11
29,Q7,Q7.2,0.56,0.58
30,Q7,Q7.3,0.76,0.40


,block,question,p,r_pb
3,Q3,Q3.4,0.78,0.05
10,Q4,Q4.4,0.61,0.40
20,Q5,Q5.7,0.65,0.17
22,Q6,Q6.2,0.65,0.42
29,Q7,Q7.2,0.56,0.58
36,Q8,Q8.2,0.78,0.43
43,Q9,Q9.2,0.61,0.05
53,Q10,Q10.5,0.78,0.37
